### Symbolic resourse estimation

All the operations below take single QFixed register (num_qubits=$n$, radix=$r$)
as input and write result in allocated QFixed register with the same size and radix.

Uncomputation is not counted.

In [3]:
import numpy as np
import sympy

from qmath.utils.re_utils import re_symbolic_fixed_point
from qmath.poly.piecewise import EvalFunctionPPA
from qmath.func.inv_sqrt import InverseSquareRoot



def _normalize(expr):
    # Replace "Max" functions with sympy.Max, so refine works.
    def _fix(e):
        if isinstance(e, sympy.Function) and e.func.__name__ == "Max":
            return sympy.Max(*map(_fix, e.args))
        return e.func(*map(_fix, e.args)) if e.args else e
    expr = _fix(expr)
    
    # Refine assuming n>=min_val, r>=min_val.
    # Idea: replace each var_i -> var_i'+min_val, simplify assuming var_i'>=0,
    # then replace var_i' -> var_i-min_val.
    min_val = 2
    syms = {s: sympy.Symbol(s.name + "'", nonnegative=True) for s in expr.free_symbols}
    expr = expr.subs({k: v + min_val for k, v in syms.items()})
    expr = sympy.refine(expr)
    expr = expr.subs({v: (k - min_val) for k, v in syms.items()})
    
    # Rename radix->r.
    vars = {s.name: s for s in expr.free_symbols}
    n, radix = vars["n"], vars["radix"]
    r = sympy.symbols("r")
    expr = expr.subs({radix: r})

    # Expand and collect as polynomial of n and r.
    expr = sympy.expand(expr)
    expr = sympy.Poly(expr, n, r).as_expr()
    return expr

OPS = [
    ("InvSqrt(iters=1)", InverseSquareRoot(num_iterations=1)),
    ("InvSqrt(iters=2)", InverseSquareRoot(num_iterations=2)),
    ("InvSqrt(iters=3)", InverseSquareRoot(num_iterations=3)),
    ("InvSqrt(iters=4)", InverseSquareRoot(num_iterations=4)),
    ("sin([-1,1], deg=3, tol=1e-8)", EvalFunctionPPA(lambda x: np.cos(x), interval=[-1, 1], degree=3, error_tol=1e-8)),
    ("sin([-1,1], deg=4, tol=1e-8)", EvalFunctionPPA(lambda x: np.cos(x), interval=[-1, 1], degree=4, error_tol=1e-8)),
    ("sin([-1,1], deg=5, tol=1e-8)", EvalFunctionPPA(lambda x: np.cos(x), interval=[-1, 1], degree=5, error_tol=1e-8)),
]
 
for name, op in OPS:
    re = re_symbolic_fixed_point(op)
    print(name)
    print("Qubits:       ", _normalize(re['qubit_highwater']),)
    print("Toffoli:      ", _normalize(re['toffs']),)
    print("Left elbows:  ", _normalize(re['gidney_lelbows']),)
    print("Right elbows: ", _normalize(re['gidney_lelbows']),)
    print("Avtive volume:", _normalize(re['active_volume']),)
    if hasattr(op, "poly"):
        print("Number of pieces:", len(op.poly.pieces))
    print()
 

InvSqrt(iters=1)
Qubits:        9*n + 2*r + 2
Toffoli:       1.5*n**2 + 3.0*n*r + 26.5*n + 1.5*r**2 + 2.5*r - 23.0
Left elbows:   1.5*n**2 + 3.0*n*r + 26.5*n + 1.5*r**2 - 0.5*r - 51.0
Right elbows:  1.5*n**2 + 3.0*n*r + 26.5*n + 1.5*r**2 - 0.5*r - 51.0
Avtive volume: 177*n**2 + 354*n*r + 2774*n + 162*r**2 + 39*r - 3680

InvSqrt(iters=2)
Qubits:        13*n + 2*r + 2
Toffoli:       3.0*n**2 + 6.0*n*r + 53.0*n + 3.0*r**2 + 5.0*r - 46.0
Left elbows:   3.0*n**2 + 6.0*n*r + 52.0*n + 3.0*r**2 - 1.0*r - 102.0
Right elbows:  3.0*n**2 + 6.0*n*r + 52.0*n + 3.0*r**2 - 1.0*r - 102.0
Avtive volume: 354*n**2 + 708*n*r + 5496*n + 324*r**2 + 78*r - 7360

InvSqrt(iters=3)
Qubits:        17*n + 2*r + 2
Toffoli:       4.5*n**2 + 9.0*n*r + 79.5*n + 4.5*r**2 + 7.5*r - 69.0
Left elbows:   4.5*n**2 + 9.0*n*r + 77.5*n + 4.5*r**2 - 1.5*r - 153.0
Right elbows:  4.5*n**2 + 9.0*n*r + 77.5*n + 4.5*r**2 - 1.5*r - 153.0
Avtive volume: 531*n**2 + 1062*n*r + 8218*n + 486*r**2 + 117*r - 11040

InvSqrt(iters=4)
Qubits: 